# Overview

This a notebook that inspects the results of a WarpX simulation.

# Instructions

Execute the cells below one by one, by selecting them with your mouse and typing `Shift + Enter`

In [ ]:
# Import statements
import sys
from tqdm import tqdm
import yt, glob
yt.funcs.mylog.setLevel(50)
from IPython.display import clear_output
import numpy as np
from ipywidgets import interact, RadioButtons, IntSlider
import matplotlib.pyplot as plt
%matplotlib

# Find iterations
file_list = glob.glob('plt?????')
iterations = [ int(file_name[3:]) for file_name in file_list ]

# Functions to plot the fields

In [ ]:
def plot_field( iteration, field, slicing_direction='y', plotter='matplotlib' ):
    ds = yt.load( './plt%05d/' %iteration )
    all_data_level_0 = ds.covering_grid(level=0, 
                        left_edge=ds.domain_left_edge, dims=ds.domain_dimensions)
    
    if plotter == 'yt':
        sl = yt.SlicePlot(ds, slicing_direction, field)
        sl.set_log( field, False)
        sl.annotate_grids()
        # Show the new plot
        clear_output()
        sl.show()

    elif plotter == 'matplotlib':

        left_edge = ds.domain_left_edge.convert_to_mks()*1.e6
        right_edge = ds.domain_right_edge.convert_to_mks()*1.e6
        
        if slicing_direction == 'x':
            n = int( ds.domain_dimensions[0]//2 )
            data2d = all_data_level_0[field][n, :, :]
            extent = [ left_edge[2], right_edge[2], left_edge[1], right_edge[1] ]
        elif slicing_direction == 'y':
            n = int( ds.domain_dimensions[1]//2 )
            data2d = all_data_level_0[field][:, n, :]
            extent = [ left_edge[2], right_edge[2], left_edge[0], right_edge[0] ]
        elif slicing_direction == 'z':
            n = int( ds.domain_dimensions[2]//2 )
            data2d = all_data_level_0[field][:, :, n]
            extent = [ left_edge[1], right_edge[1], left_edge[0], right_edge[0] ]
        plt.clf()
        plt.title("%s at iteration %d" %(field, iteration) )
        plt.imshow( data2d, interpolation='nearest', cmap='viridis',
                  origin='lower', extent=extent, aspect='auto' )
        plt.colorbar()

# Interactive viewer

In [ ]:
interact(plot_field, 
         iteration = IntSlider(min=min(iterations), max=max(iterations), step=iterations[1]-iterations[0]),
         field = RadioButtons( options=['jx', 'jy', 'jz', 'Ex', 'Ey', 'Ez'], value='jz'),
         slicing_direction = RadioButtons( options=[ 'x', 'y', 'z'], value='y'),
         plotter = RadioButtons( options=['matplotlib', 'yt'] ) )